In [1]:
# import our chat-bot intents file
import json
import pickle
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
#with open('intents.json') as json_data:
#    intents = json.load(json_data)

Using TensorFlow backend.


In [2]:
from flask import Flask,jsonify
from flask import make_response
from flask import abort
from flask import request
app = Flask(__name__)

In [3]:
@app.route('/api/intent/all/', methods=['GET'])
def  getIntentResponse():
    # Read all intent and return
    with open('intents.json') as json_data:
        intents = json.load(json_data)
    
    return jsonify({'all_intents':intents})

In [4]:
@app.route('/api/intent/response/<intent_id>', methods=['GET'])
def  getTagResponse(intent_id):
    
    # Read all intent and return
    with open('intents.json') as json_data:
        intents = json.load(json_data)
    
    # variable that will hold the response
    resp = ''
    
    # Loop over all intent and find the tag
    for intent in intents['intents']:
        for intent_tag in intent['tag']:
            if intent_tag == intent_id:
                resp = intent['responses']
    
    # If not intent found throw error
    if len(resp) == 0:
        abort(404)
        
    return jsonify({'response':resp})

In [5]:
@app.route('/api/reviews/<review_text>', methods=['GET'])
def  getPredictionReview(review_text):
    
    
    userReview = review_text
    
    print('User review is :'+userReview )

       
    return jsonify({'response':userReview})

In [6]:
# loading
@app.route('/api/review/<id>', methods=['GET','POST'])
def  getReviewPrediction(id):
    
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer_saved = pickle.load(handle)

    # Returns a compiled model identical to the previous one
    model_saved = load_model('my_model.h5')
    
    content = request.json
    review_sen = content['userReview']
    #review_sen = "wow service"

    tokens_sen = tokenizer_saved.texts_to_sequences([review_sen])
    pad_tokens_sen = pad_sequences(tokens_sen, padding='post', maxlen=100)
    
    val = model_saved.predict_classes(pad_tokens_sen)
    
    
    if (val[0][0] == 0):
        reviewSentiment = 'Customer is gone forever,'
    else:
        reviewSentiment = 'you got back your customer'

    return reviewSentiment;

In [7]:
@app.route('/api/message/<msg_id>', methods=['GET', 'POST'])
def add_message(msg_id):
    content = request.json
    print (content['mymsg'])
    return jsonify({"msg id":msg_id})


In [8]:
@app.errorhandler(404)
def not_found(error):
    return make_response(jsonify({'error': 'Not found'}), 404)

In [ ]:
# Run the application
if __name__ == '__main__':
    app.run(host='127.0.0.1', port=21021)

 * Running on http://127.0.0.1:21021/ (Press CTRL+C to quit)
127.0.0.1 - - [05/Oct/2019 12:14:39] "POST /api/review/4 HTTP/1.1" 200 -
127.0.0.1 - - [05/Oct/2019 12:14:46] "POST /api/review/4 HTTP/1.1" 200 -
127.0.0.1 - - [05/Oct/2019 12:14:54] "POST /api/review/4 HTTP/1.1" 200 -
